In [66]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as BS
import jieba
import pymysql
import re

In [3]:
df=pd.read_csv('中国地区代码表.csv',encoding='gbk')

In [23]:
plist=[]
for i in range(len(df)):
    plist.append(df.loc[i,'名称'].split('--')[-1])

In [24]:
plist=sorted(list(set(plist)))
plist2=[]
for p in plist:
    if len(p)>2:
        plist2.append(p[:-1])
plist+=plist2
plist=sorted(list(set(plist)))
plist=plist[8:]

In [34]:
#获取百度百科行业类别表
headers = {
    'Connection': 'close',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
res=requests.get("https://baike.baidu.com/item/%E5%9B%BD%E6%B0%91%E7%BB%8F%E6%B5%8E%E8%A1%8C%E4%B8%9A%E5%88%86%E7%B1%BB/1640176?fr=aladdin",headers=headers)
res.encoding='utf8'
soup=BS(res.text,'lxml')

In [43]:
dflb=pd.DataFrame(columns=['门类','大类','中类','小类','类别名称'])
n=0
col=['门类','大类','中类','小类','类别名称']
for tr in soup.table.find_all('tr')[2:]:
    tds=tr.find_all('td')
    for i in range(5):
        dflb.loc[n,col[i]]=tds[i].text.strip()
    n+=1

In [46]:
dflb.to_csv('行业分类.csv',index=False)

In [48]:
llist=dflb.loc[:,'类别名称'].tolist()
lbc=[]
for l in llist:
    lbc+=jieba.lcut(l)
lbc=sorted(list(set(lbc)))

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\sigmeta\AppData\Local\Temp\jieba.cache
Loading model cost 0.924 seconds.
Prefix dict has been built succesfully.


In [50]:
lb=[]
for i in lbc:
    if len(i)>1:
        lb.append(i)

In [78]:
stopwords=plist+lb+['公司','有限公司','集团','集团股份','股份','总公司','控股','实业','有限责任','检测']

In [54]:
#连接数据库
conn = pymysql.connect('localhost','root','9432','companyb',charset='utf8')
cursor=conn.cursor()

In [55]:
cursor.execute("select * from com_inv")
a=cursor.fetchall()

In [74]:
jieba.lcut('七台河市东方博达产业投资发展有限责任公司')

['七台河市', '东方', '博达', '产业', '投资', '发展', '有限责任', '公司']

In [79]:
for i in range(600,700):
    tag=True
    s1=re.sub('（.*）','',a[i][0]);s2=a[i][1]
    c1=jieba.lcut(s1)
    c2=jieba.lcut(s2)
    count=0
    countn=0
    for c in c1[:]:
        if c not in stopwords and c in c2:
            tag=False
            break
    print(tag,s1,s2)

True 万合集团邯郸华骏汽车检测有限公司 邯郸市复兴区欣驰汽车维护有限公司
True 万合集团邯郸华骏汽车检测有限公司 邯郸市安骏机动车检测有限公司
False 万合集团邯郸天润房地产开发有限公司 五指山万合房地产开发有限公司
False 万合集团邯郸天润房地产开发有限公司 五指山万合房地产开发有限公司
False 万合集团邯郸天润房地产开发有限公司 河北万合汽车贸易股份有限公司
False 万向三农集团有限公司 万向德农股份有限公司
True 万向三农集团有限公司 德华集团控股股份有限公司
False 万向三农集团有限公司 北京万向德农肥业有限公司
True 万向三农集团有限公司 杭州品向位食品有限公司
False 万向三农集团有限公司 万向财务有限公司
True 万向三农集团有限公司 浙江网商银行股份有限公司
True 万向三农集团有限公司 浙江省远洋渔业集团有限公司
True 万向三农集团有限公司 万事利集团有限公司
True 万向三农集团有限公司 杭州万事利丝绸文化股份有限公司
True 万向三农集团有限公司 浙江大洋世家股份有限公司
True 万向信托有限公司 浙江另类资产管理有限公司
True 万向信托有限公司 浙信投资（浙江）有限公司
True 万向信托有限公司 深圳市华美达一号投资中心（有限合伙）
True 万向信托有限公司 济南中建泉慧投资运营有限公司
True 万向信托有限公司 海宁向鼎影视传媒有限公司
True 万向信托有限公司 广东公明资产管理股份有限公司
True 万向信托有限公司 浙江农科种业有限公司
True 万向信托有限公司 深圳市汇盛投资合伙企业（有限合伙）
True 万向信托有限公司 共青城杰之行投资中心（有限合伙）
True 万向信托有限公司 宁波嘉业汇投资管理有限公司
True 万向创业投资股份有限公司 浙江联强数控机床股份有限公司
True 万向创业投资股份有限公司 浙江京昌电子股份有限公司
True 万向创业投资股份有限公司 浙江力宝高新建材股份有限公司
True 万向创业投资股份有限公司 北京新意互动广告有限公司
True 万向创业投资股份有限公司 杭州先临三维科技股份有限公司
True 万向创业投资股份有限公司 北京北纬通信科技股份有限公司
True 万向创业投资股份有限公司 深圳市贝特瑞新能源材料股份有限公司
Tr

In [ ]:
for i in range(len(a)):
    tag=True
    s1=re.sub('（.*）','',a[i][0]);s2=a[i][1]
    c1=jieba.lcut(s1)
    c2=jieba.lcut(s2)
    count=0
    countn=0
    for c in c1[:]:
        if c not in stopwords and c in c2:
            tag=False
            break
    if tag:
        try:
            cursor.execute("insert into com_inv_1 values ('%s', '%s', '%s', '%s', '%s');"%a[i])
        except Exception as e:
            print(e)  
    else:
        print('xx',a[i])
    if i%1000==0:
        conn.commit()
conn.commit()